In [ ]:
from nbtemplate import display_header, get_path
display_header('Chirp.ipynb')

# Chirped gratings

*Chirp is a change of grating period over a grating. This can be used to compensate abberations from a finite grating size. In this notebook, I determine the magnitude of the chirp needed for the Lynx XGS. For simplicity, we do not want to manufacture gratings with individual chirps, so I test how many different types of gratings are needed to match Lynx XGS resolving power requirements. I find that having just a three different types of gratings is sufficient to hit the Lynx XGS resolving power requirements, while reducing the number of gratings by an order of magnitude and increasing the effective area by a quarter compared to the baseline design using 20 * 50 mm gratings. Bending the gratings to reduce the ditribution of blaze angles improves the effective area by another 10% with a simultaneous gain in resolving power.*

## Introduction

The Rowland torus is curved, but transmission gratings are usually flat. Thus, for any positioning of gratings, parts of the grating will be above and others below the Rowland torus surface. If a ray intersects with a grating too early, the ray will be diffracted at a larger distance from the focal plane and thus will land further away from the zeroth order, while rays intersecting a grating below the surface of the Rowland torus are diffracted not as far. This effect spreads out the photons and reduces the spectral resolving power. In earlier X-ray instruments with transmission gratings, e.g. the Chandra/HETG, this is not a limiting effect. Gratings are small (about 20 * 20 mm or so) and the resolving power requirements are relatively modest compared with Lynx. However, for the Lynx X-ray grating spectrograph (XGS) this is actually an effect that can limit resolving power and drives the choice of grating size in the baseline design.

Different approaches can be used to compensate the loss in resolving power. The simplest idea might be to just use smaller gratings. As shown in the [notebook on subaperturing](../Subaperturing.html) gratings of 20*50 mm are small enough to match Lynx resolvng power requirements. However, several thousand gratings are needed to cover the aperture in this case. A second idea is to bend the gratings so that they are not flat and follow the torus better. Initial tests are promising for bend gratings. [Heine et al. (2017)](https://doi.org/10.1117/12.2274205) show that bending does not change their diffration efficiency and do not report macroscopic damage, but tests of resolving power in X-rays have not been done yet and the grating used for the bending experiement was only 10 mm * 30 mm in size. So, in priciple curved gratings can be used to match the grating position better to the Rowland torus. This is complicated by the fact that we also try to maintain a narrow range of blaze angles to optimize the number of photons diffracted into grating orders which are covered by detectors. The radius of the Rowland circle is about half of the focal distance. To maintain the ideal blaze angle for all rays in a converging beam, the gratings would have to be bent with a radius close to the focal length, while the radius to match the surface of the torus is half of that. To make matters worse, in CAT (critical angle transmission) gratings, the gratings bars must be inclined with repect to the incoming rays. The structure of the silicon crystal and the CAT grating manufacturing process require that the gratings bars are perpendicular to the surface of the grating. In order to maintain a blaze angle around 1.6 degrees, gratings have to be mounted with an angle, adding further restrictions on bending them.

So, another appraoch is to use flat gratings, oriented to make the blaze condition work at least for the center ray, and to change the grating period $d$ smoothly over the grating ("chirp") such that the part of the grating that is located "below" the surface of the Rowland torus, where photons would be diffracted not far enough, has a slightly smaller $d$ and thus diffract to larger angles. Conversely, the part of the grating located "above" the Rowland torus needs to have a slightly larger $d$. The grating equation gives the relation between $\lambda$, the wavelength of the photon, the grating order $n$, and the angle of diffraction $\theta$:
$$
n\lambda = d \sin \theta \; .
$$

In [ ]:
from marxslynx.lynx import PerfectLynx, conf_chirp

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from marxs.source import PointSource, FixedPointing
from marxs.simulator import Sequence
from marxs.analysis.gratings import (resolvingpower_from_photonlist,
                                     effectivearea_from_photonlist)
from marxslynx.bendgratings import NumericalChirpFinder, chirp_gratings
from marxs.optics import CATGrating, CircularDetector

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import copy
# make a configuration that's similar to the chirp setup, but do not
# chip gratings in the initialization routine. Instead, that's done with 
# several different settings below.
conf_nochirp = copy.copy(conf_chirp)
del conf_nochirp['chirp_energy']
del conf_nochirp['chirp_order']

In [ ]:
insdefault = PerfectLynx()
#ins5050 = PerfectLynx(conf_5050)
insnochirp = PerfectLynx(conf_nochirp)
inschirp = PerfectLynx(conf_chirp)


## General setup
The baseline design for the Lynx XGS uses gratings of 20 mm * 50 mm, where the long direction is perpendicular to the dispersion and is chosen to minimze technical risk, as this size of close to the sie of gratings that have been manufactured in the past. The short side is along the dispersion direction and is limited by the aberration discussed above. At this size, over 5000 gratings are required to cover the aperture. As discussed in the [notebook on subaperturing](../Subaperturing.html), for most of the aperture, larger gratings of 50 mm * 50 mm are also sufficient. So, with the added complexity of having gratings of two different sizes, the number of gratings can be brought down to about 3000. In contrast, we here investigate much larger gratings of 60 mm * 180 mm. At this size, only about 500 gratings are required to cover the full aperture. The choice of 60 mm * 180 mm is driven by the size of commercially available 200 mm waivers, such that one grating fits onto a waiver with comfortable margin for handling and processing.

In the following simulations, we assume that the aperture is completly filled with gratings. For the baseline design, that is not necessary as 2/3 of the aperture already provide enough effective area to fullfill the science requirements. That allows us to leave some part of the aperture open and improve the resolving power through sub-aperturing, while simultaneously enhancing the count rate in zeroth order. To simplify the following discussion, we concentrate on completely filled apertures. Thus, the resolving power $R$ shown here is slightly below the values given for the Lynx design reference mission (DRM), while the effective area $A_{\mathrm{eff}}$ is above it.
Similarly, we simplify the discussion by assuming a continous detector with no chip gaps. This is just a computational simplification, such that we can run simulations for just three wavelength points without worrying if any one of them falls into a chip gap.

Ray-racing is run with $10^5$ rays per simulation.

In [ ]:
# Check size of default gratings
from transforms3d.affines import decompose44
decompose44(insdefault.elements[2].elements[0].geometry.pos4d)[2]

In [ ]:
len(insdefault.elements_of_class(CATGrating)), len(inschirp.elements_of_class(CATGrating))

## Approach
We chose a reference energy in the middle of the XGS bandpass (0.6 keV). 
For each grating, we calculate the position on the detector, where a ray that passes through the center of the grating intersects with the detector. For all other positions on the detector, we can then determine which diffraction angle $\theta$ is required to bring rays to the same position on the detector. From the grating equation, we can determine the value of $d$ required at that grating position. The 3D geometry of this is a litte cumbersome, so for simplicity we optimize $d$ numerically instead of tracing the geometry analytically. Numerically, we just perform the ray-trace varying $d$. Since we perform a ray-trace anyway, this is simple and fast to implement.

Of course, the diffraction of light requires a periodic structure and would not work if gratings did not have a regular period. However, the required chirp is so small that the period $d$ changes noticably only over macroscopic scales, thus diffraction still occurs. This is somewhat analogous to stacking a telescope with a large number of smaller gratings. The grating bars are periodic within each grating, but the gratings are not phased up, and diffraction still happens, as evidenced by the Chandra/HETGS and other observatories.

In principle, the chirp in $d$ is not the same for all wavelength and all angles, but since CAT gratings diffract photons into a relatively narrow range of angles, that is not a problem in practice, as we show below.

In [ ]:
from marxslynx.ralfgrating import facet_table
facetpos = facet_table(insnochirp.elements[2])

In [ ]:
opt = NumericalChirpFinder(insnochirp.elements_of_class(CircularDetector)[0], 
                           order=-5.4, energy=0.6 * u.keV, d=0.0002)

uarray = np.linspace(-.999, .999, 50)
varray = np.array([0])
catgratings = insnochirp.elements_of_class(CATGrating)
# select a subset to speed up experiementing
gratings = catgratings[::100]
corr = np.zeros((len(gratings), len(uarray)))
for i, grat in enumerate(gratings):
    corr[i, :] = opt(grat, uarray, varray)[:, 0]


In [ ]:
fig, ax = plt.subplots()
out = ax.plot(uarray * conf_chirp['gas_kwargs']['elem_args']['zoom'][1], corr.T - 1)
ax.set_xlabel('position on grating [mm]')
ax.set_ylabel('fractional change in\ngrating period')
fig.savefig(get_path('figures') + '/chirps.png', dpi=300)
fig.savefig(get_path('figures') + '/chirps.pdf', bbox_inches='tight')

The figure above shows the relative change in the grating period for different gratings as calculated numerically. The gratings are located at different locations on the aperture. Some of them need a decreasing grating period in the direction of the positive y-axis, other require an increasing period. What they have in commmon though, is that all solutions are very close to linear. Thus, from now on, I will impose a linear relationship between the position on the grating and the fractional change in period. This speeds up the calcualtions significantly, because only a single point is needed to define a line that goes through 0 at the center of the ratings. Beyond numerical convenience, this also simplifies the specification of actual phyiscal gratings. While a different gradient in grating period $d$ might be needed for different gradients, all chirps have a linear relationship.

### How many different types of grating do we need?

The ideal chirp is different for every single grating. However, from a cost and schedule perspective, it is much preferrable to use only few different types of grating. We can manufacture gratings with a few different chirps and then for each position pick a chirp that is very close to the ideal number. This deviation will lead to some loss of resolving power, but this is acceptable to a certain degree. While the simulations here are done with a perfectly aligned model, in practice, mechanical misalginments, aberations, and pointing errors all reduce the theoretically achievable resolving power. A small contribution from using grating with non-ideal chirps does not impact the final resolving power significantly.

In this section, I test how many different grating types are needed. I assume that there will always be a group of gratings with no chirp. For example, a simulation might use gratings with just three different chirps: -0.0004, 0, 0.0004. Here the value of the chirp is expressed as the relative change of the grating period from the center to one edge. The relative change in $d$ for chirps of -0.0004 and +0.0004 is the same, just in one case it is increasing from left to right, in the other case it is decreasing. CAT gratings do not have a preferred direction and the same type of grating can be used, just mounted with a different rotation, such that three values of the chirp can be realized with just two different physical gratings (or, e.g., five values of the chirp with just three types of grating, one with constant $d$ and two with different chirps). In practice, grating facets will have some handling layer, screw holes for mounting or similar, so that this has to be taken into account in the mechnical design, but this is simpler than manufacturing, testing, etc. more different grating types.

In [ ]:
def set_linear_chirp(grating, d, rel_chirp):
    '''
    Parameters
    ----------
    grating : `marxs.optics.grating.CATGrating`
    d : float
        Grating period
    rel_chirp : 
        relative change of grating period (relative to center) at the outermost +y
        position in the grating. If e.g. the chirped period range from 0.99 to 1.01, then
        ``rel_chirp = 0.01``.
    '''
    def func(self, intercoos):
        ly = np.linalg.norm(self.geometry['v_y'])
        return d * (1 + intercoos[:, 0] / ly * rel_chirp)
    grating._d = func.__get__(grating)

In [ ]:
basenochirp = PerfectLynx(conf_nochirp)
catgratings = insnochirp.elements_of_class(CATGrating)

linchirp = np.zeros(len(catgratings))
for i, grat in enumerate(catgratings):
    linchirp[i] = opt(grat, np.array([0.9999]), np.array([0]))
    grat.order_selector = grat.original_orderselector


In [ ]:
fig, ax = plt.subplots()
scat = ax.scatter(facetpos['facet_y'], facetpos['facet_z'], c=linchirp - 1)
cbar = plt.colorbar(scat, ax=ax)
ax.set_xlabel('dispersion direction [mm]')
ax.set_ylabel('cross-dispersion direction [mm]')

In [ ]:
plt.plot(facetpos['facet_y'], linchirp -1, '.')
plt.grid()
plt.xlabel('dispersion direction [mm]')
plt.ylabel('chirp [relative between center and edge of grating]')

The two figures above show how the required chirp depends on the position. *top:* The 2D view shows that there is no dependence on the cross-dispersion direction, only the dispersion direction matters. *bottom:* The dependance on the $y$ coordinate is almost linear, but is not fully symmetric since the line does not pass through (0, 0). 

In [ ]:
def make_instrum_discretechirp(chirp):
    instrum = PerfectLynx(conf_nochirp)

    gratchirps = np.array(chirp)
    ind = np.argmin(np.abs((linchirp - 1) - gratchirps[:, None]), axis=0)
    chosen_chirp = np.choose(ind, gratchirps)
    for i, grat in enumerate(instrum.elements_of_class(CATGrating)):
        set_linear_chirp(grat, conf_nochirp['gas_kwargs']['elem_args']['d'], chosen_chirp[i])
    # The chosen is output only for debugging. Can delete htat from return once it works
    return instrum, chosen_chirp

In [ ]:
chirp2, chosen2 = make_instrum_discretechirp([-0.0004, 0, 0.0004])
chirp3, chosen3 = make_instrum_discretechirp([-0.0005, -0.00025, 0, 0.00025, 0.0005])

# Could make this more general, but if I only use it for a few cases, than faster this way
c5 = np.linspace(0, 0.0007, 5)
chirp5, chosen5 = make_instrum_discretechirp(np.hstack([-c5[::-1], c5[1:]]))

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, sharex=True, sharey=True,
                         subplot_kw={'aspect': 'equal'},
                        figsize=(6,6))

data = [linchirp - 1, chosen2, chosen3, chosen5]
labels = ['individual chirp', '2 grating types', '3 grating types', '5 grating types']

for i, ax in enumerate(axes.flatten()):
    scat = ax.scatter(facetpos['facet_y'], facetpos['facet_z'], c=data[i], 
                      vmin=np.min(linchirp - 1), vmax=np.max(linchirp - 1))
    #cbar = plt.colorbar(scat, ax=ax)
    ax.set_title(labels[i])
axes[1, 0].set_xlabel('dispersion direction [mm]')
axes[1, 1].set_xlabel('dispersion direction [mm]')
axes[0, 0].set_ylabel('cross-dispersion direction [mm]')
axes[1, 0].set_ylabel('cross-dispersion direction [mm]')
cax = fig.add_axes([.82, .1, .03, .8])
cbar = plt.colorbar(scat, cax=cax)
cbar.set_label('period change on edge of grating relative to center')
fig.subplots_adjust(right=.8)
fig.savefig(get_path('figures') + '/chirps_distribution.png', dpi=300)
fig.savefig(get_path('figures') + '/chirps_distribution.pdf', bbox_inches='tight')

This figure shows the distribution of chirps for different simulations. *top left:* Gratings have individual chirps. *other panels:* Setups with a limited number of grating types where for each position a chrip is chosen that is as close to the ideal chirp at that position as available. Assuming gratings can be reversed, two different chirps ($0, c$) allow for three different values ($-c, 0, c$) in the figure. In the case with five grating types (bottom right) gratings with the largest chirp are only found on one side, so the number of strips is 8, not 9 in that panel. This is because the chirp scale is not symmetric (the colorbar ranges from about -0.0007 to + 0.0005) and gratings with the largest chirp are simply not needed in the positive dispersion direction. Note that gratings do not touch each other, but at this scale the space between them is hard to see in the figure.

In the following, we show simualtions for the four scenarios shown in the figure and for comparison also for our baseline XGS desgin (no chirp, grating size 20 mm * 50 mm) and for a desgin with gratings of the same size (80 mm * 160 mm) as the four chirped scenarios, but no chirp.

In [ ]:
orders = np.arange(-8, 1)
energy = [0.4, 0.6, 0.8] * u.keV
inslist = [insdefault, insnochirp, inschirp, chirp2, chirp3, chirp5]
inslabels = ['20 * 50 mm', 'no chirp', 'individual chirp', '2 grating types', '3 grating types', '5 grating types']

In [ ]:
def run_chirps(inslist, energy=energy, orders=orders):

    resolvingpower = np.zeros((len(inslist), len(energy), len(orders)))
    effective_area = np.zeros_like(resolvingpower)
    plist = []

    for i, e in enumerate(energy):
        mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                               energy=e,
                               flux=1. / u.s / u.cm**2)
        fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
        photons = mysource.generate_photons(1e5 * u.s)
        photons = fixedpointing(photons)
        # mirrors are the same for each instrument, so run only once to save time
        photons = Sequence(elements=inslist[0].elements[:2])(photons)

        for j, ins in enumerate(inslist):
            p = Sequence(elements=ins.elements[2:])(photons.copy())
            plist.append(p)
    return plist

In [ ]:
thetalimits = np.array(conf_chirp['det_kwargs']['theta']) - np.pi

def res_aeff(plist, inslist, thetalimits=thetalimits, energy=energy, orders=orders):
    
    resolvingpower = np.zeros((len(inslist), len(energy), len(orders)))
    effective_area = np.zeros_like(resolvingpower)

    for i, e in enumerate(energy):
        for j, ins in enumerate(inslist):
            p = plist[i * len(inslist) + j]
            res, width, pos = resolvingpower_from_photonlist(p, orders, col='detcirc_phi')
            resolvingpower[j, i, :] = res
            ind = (p['detcirc_phi'] > thetalimits[0]) & (p['detcirc_phi'] < thetalimits[1])
            effective_area[j, i, :] = effectivearea_from_photonlist(p[ind], orders,
                                                                    len(p), ins.elements[0].area)
    return resolvingpower, effective_area    

In [ ]:
def summarize(resolvingpower, effective_area):
    resolvingpower = np.ma.masked_invalid(resolvingpower)
    # Mask out the zeros order which always has resolving power 0 
    resolvingpower[:, :, orders==0] = np.ma.masked

    # Extra copy while I debug the code, so I don't have to re-run every time
    res = np.ma.average(resolvingpower.copy(), axis=-1, weights=effective_area)
    aeff = np.ma.masked_invalid(effective_area)
    aeff[:, :, orders==0] = np.ma.masked
    return res, aeff

In [ ]:
plist = run_chirps(inslist)
resolvingpower, effective_area = res_aeff(plist, inslist)
res, aeff = summarize(resolvingpower, effective_area)

In [ ]:
for i in range(6):
    plt.plot(orders, resolvingpower[i, 0, :], label=inslabels[i])
plt.legend()
plt.ylabel('Resolving power')
out = plt.xlabel('difrraction order')

This figure shows how the resolving power changes with spectral order. Most of the signal is concentrated in only a few orders, so below we summarize the numbers shown here by calculating the average resolving power, weighted by the number of photons in each order.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
for i in range(len(inslabels)):
    axes[0].plot(energy, res[i, :], label=inslabels[i], lw=2)
    # 100 converts from mm^2 to cm^2
    axes[1].plot(energy, aeff[i, :, :].sum(axis=-1) / 100, label=inslabels[i], lw=2)

for ax in axes:
    ax.set_xlabel('energy [keV]')
axes[0].set_ylim(0, None)
    
axes[1].legend()
axes[0].set_ylabel('Resolving power')
axes[1].set_ylabel('effective area [cm$^2$]')
fig.subplots_adjust(wspace=.3)
fig.savefig(get_path('figures') + '/chirp_aeff_res.png', dpi=300)
fig.savefig(get_path('figures') + '/chirp_aeff_res.pdf', bbox_inches='tight')

Resolving power (left) and effective area (right) for different grating arrangements. The resolving power $R$ given is averaged over all dispersed orders, weighted by the detected signal in each order. Simulations are performed for three energies, spanning the XGS bandpass. 

The scenarios shown include the baseline design with 20 mm * 50 mm gratings without any chirp, 80 mm * 160 mm gratings without chrip and chirped gratings, with either an individual chirp for each grating, or with just 2, 3, or 5 differeny types of gratings. The baseline scenario matches the Lynx requirements on $R$ and effective area, although a significant fraction of the aperture is covered by the frames and holders located around the grating membrane. The right plot shows that the effective area increases by about 20% when larger gratings are used. The exact number depends on the distribution of the gratings over the aperture, the position of the mirror support spider, the size of the grating holders, the mounting strucutre etc, but it is clear that larger gratings offer a better $A_{\mathrm{eff}}$, while simultaneously being more cost-effective because far fewer elements need to be manufactured, tested, and installed. The increased $A_{\mathrm{eff}}$ can either be used to improve the science output, or, for constant $A_{\mathrm{eff}}$ compared to the baseline design, we could subaperture more, increasing $R$ over the number shown in the figure and reducing the number of gratings needed even further. 

In all cases, the chirp is calculated for photons of 0.6 keV, but for the angles and diffraction orders relevant for the XGS, the chirp is equally effective for all relevant energies.



## Bending chirped gratings

In [ ]:
from marxslynx.bendgratings import bend_gratings
from marxs.missions.mitsnl.catgrating import CATL1L2Stack

bend = PerfectLynx(conf_nochirp)
bend_gratings(bend.elements_of_class(CATL1L2Stack), r=9500)


In [ ]:
from marxslynx.bendgratings import BendNumericalChirpFinder

optbend = BendNumericalChirpFinder(insnochirp.elements_of_class(CircularDetector)[0], 
                           order=-5.4, energy=0.6 * u.keV, d=0.0002)

In [ ]:
uarray = np.linspace(-.999, .999, 50)
varray = np.array([0])
catgratings = bend.elements_of_class(CATGrating)
# select a subset to speed up experiementing
gratings = catgratings[::100]
corr = np.zeros((len(gratings), len(uarray)))
for i, grat in enumerate(gratings):
    corr[i, :] = optbend(grat, uarray, varray)[:, 0]

In [ ]:
fig, ax = plt.subplots()
out = ax.plot(uarray * conf_chirp['gas_kwargs']['elem_args']['zoom'][1], corr.T - 1)
ax.set_xlabel('position on grating [mm]')
ax.set_ylabel('fractional change in\ngrating period')
fig.savefig(get_path('figures') + '/chirpsbend.png', dpi=300)
fig.savefig(get_path('figures') + '/chirpsbend.pdf', bbox_inches='tight')

This figure shows the change in the fractional grating period that is required to bring photons that intersect a bend grating in a position at some distance from the grating center (so gratings are bent first and then the chirp is calculated for the bend grating). As above for flat gratings, the dependence is linear. 

In [ ]:
def set_bend_chirp(grating, d, rel_chirp):
    '''
    Parameters
    ----------
    grating : `marxs.optics.grating.CATGrating`
    d : float
        Grating period
    rel_chirp : 
        relative change of grating period (relative to center) at the outermost +y
        position in the grating. If e.g. the chirped period range from 0.99 to 1.01, then
        ``rel_chirp = 0.01``.
    '''
    def func(self, intercoos):
        return d * (1 + intercoos[:, 0] / grat.geometry.phi_limits[1] * rel_chirp)
    grating._d = func.__get__(grating)

In [ ]:
basenochirp = PerfectLynx(conf_nochirp)

bend = PerfectLynx(conf_nochirp)
bend_gratings(bend.elements_of_class(CATL1L2Stack), r=9500)

catgratings = bend.elements_of_class(CATGrating)
bendchirp = np.zeros(len(catgratings))
for i, grat in enumerate(catgratings):
    bendchirp[i] = optbend(grat, np.array([0.9999]), np.array([0]))
    grat.order_selector = grat.original_orderselector
    set_bend_chirp(grat, conf_nochirp['gas_kwargs']['elem_args']['d'], bendchirp[i] - 1)

In [ ]:
def make_bendinstrum_discretechirp(chirp):
    instrum = PerfectLynx(conf_nochirp)
    bend_gratings(instrum.elements_of_class(CATL1L2Stack), r=9500)

    gratchirps = np.array(chirp)
    ind = np.argmin(np.abs((bendchirp - 1) - gratchirps[:, None]), axis=0)
    chosen_chirp = np.choose(ind, gratchirps)
    for i, grat in enumerate(instrum.elements_of_class(CATGrating)):
        set_bend_chirp(grat, conf_nochirp['gas_kwargs']['elem_args']['d'], chosen_chirp[i])
    # The chosen is output only for debugging. Can delete htat from return once it works
    return instrum, chosen_chirp

In [ ]:
bchirp2, bchosen2 = make_bendinstrum_discretechirp([-0.0004, 0, 0.0004])
bchirp3, bchosen3 = make_bendinstrum_discretechirp([-0.0005, -0.00025, 0, 0.00025, 0.0005])
bchirp5, bchosen5 = make_bendinstrum_discretechirp(np.hstack([-c5[::-1], c5[1:]]))

In [ ]:
binslist = [bend, bchirp2, bchirp3, bchirp5]

In [ ]:
pblist = run_chirps(binslist)
bresolvingpower, beffective_area = res_aeff(pblist, binslist)
bres, baeff = summarize(bresolvingpower, beffective_area)
plist = run_chirps(inslist)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
lines = [] # just to grap the line colors, eaiser to getting the matplotlib default color cycle
for i in range(len(inslabels)):
    lines.extend(axes[0].plot(energy, res[i, :], label=inslabels[i], lw=2))
    # 100 converts from mm^2 to cm^2
    axes[1].plot(energy, aeff[i, :, :].sum(axis=-1) / 100, label=inslabels[i], lw=2)

for i in range(len(binslist)):
    axes[0].plot(energy, bres[i, :], lw=5, ls=':', c=lines[2 + i].get_color())
    # 100 converts from mm^2 to cm^2
    axes[1].plot(energy, baeff[i, :, :].sum(axis=-1) / 100, 
                 lw=5, ls=':', c=lines[2 + i].get_color())
    
for ax in axes:
    ax.set_xlabel('energy [keV]')
axes[0].set_ylim(0, None)
    
axes[1].legend()
axes[0].set_ylabel('Resolving power')
axes[1].set_ylabel('effective area [cm$^2$]')
fig.subplots_adjust(wspace=.3)
fig.savefig(get_path('figures') + '/chirp_aeff_res_bend.png', dpi=300)
fig.savefig(get_path('figures') + '/chirp_aeff_res_bend.pdf', bbox_inches='tight')

In the previous section, I show a figure for $R$ and $A_{\mathrm{eff}}$ for flat chirped gratings. Here, I repeat the figure and add bent chirped gratings as dashed lines. The resolving power is similar to or slightly better than for bent chirped gratings than for flat chirped gratings, in particular if more than two or three different grating types are used. However, in practice, the alignment, which is not included in these simulations, also limits the achivable resolving power. As long $R$ is larger than the minimum requirement for the Lynx XGS, this effect is not the limiting factor. This would change, if the requirement was changed to $R > 8000$ or so. In that case, the alignment requirements would have to be tightend up and the effect studied here might become the limiting factor again.

Given that the gratings are located in a converging beam, for a flat grating the blaze angle will change between the left and the right side of a grating. On the other hand, the focal length is large, so that the difference is small, even for relatively large gratings. If not all orders are detected, then bent gratings might offer an advantage, because they follow the blaze angle description better and thus disperse more photons in the orders that *are* covered by detectors. For the three energies we ray-trace here, bending the gratings increases the effective area by another 5-10\%. We look at this effect in more detail in the next figure.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 5))
photonlist = [plist[1], pblist[0], plist[0]]
histkwargs = [{'label': '80 mm * 160 mm - flat'},
              {'label': '80 mm * 160 mm - bent', 'histtype': 'step', 'lw': 3},
              {'label': '20 mm * 50 mm - flat', 'histtype': 'step', 'lw': 3, 'ec': 'k'}]

for i, p in enumerate(photonlist):
    axes[0].hist(np.rad2deg(p['blaze']), bins=np.arange(1.3, 1.9, .01),
                 weights=p['probability'], **histkwargs[i])
    axes[1].hist(p['projcirc_y'], bins=np.arange(-10, 800, 50), 
                 weights=p['probability'], **histkwargs[i])
    #print(p['probability'].sum())

axes[0].set_xlabel('Blaze angle [deg]')
axes[0].set_ylabel('Number of photons')
axes[0].legend(loc=(.1, .5))
axes[1].set_xlabel('dispersion on detector [mm]')
axes[1].set_ylabel('Number of photons')
fig.subplots_adjust(wspace=.3)
#axes[1].legend()
fig.savefig(get_path('figures') + '/blazebend.png', dpi=300)
fig.savefig(get_path('figures') + '/blazebend.pdf', bbox_inches='tight')

*left:* Distribution of blaze angles with small flat, large flat, and large bent gratings. Because of the converging beam, a range of blaze angles is unavoidable for flat gratings - the larger the grating, the larger the range. Bending the grating can compensate this effect. For simplicity of the assembly, I use the same bending radius for all gratings close to the mean distance of the gratings from the focal point. However, the gratings are arranged on the surface of the Rowland torus and thus have different distances to the focal point, so even in the case with bent gratings that is simulated here, the blaze angles show a small spread.

*right:* Distribution of photons on the detector for a 0.4 keV simulation. Individual orders are apparent. The zeroth order (direct light) is at detector position 0 mm. The brightest order is order 4 around 600 mm, which has noticably more signal in the simulation with bent gratings, while flat gratings distribute more photons into other orders.

The probability to diffract a photon into a specific diffraction order depends on the blaze (for CAT gratings, the most likely diffraction angle is about twice the the blaze angle). Because rays hitting a large flat grating in a converging beam cover a range of blaze angles, the photons are distributed more than in the case of bent gratings which maintain the design blaze angle over the entire surface of the grating. The Lynx XGS is designed with 16 detectors that cover the dispersion coordinate from about 400 to 700 mm. For photons of 0.4 keV that includes just the two most prominent orders. Bent gratings concentrate more light into these orders, and thus raise the effective area of the instrument compared to flat gratings. This is an energy dependent effect, and as we saw above, bending the gratings can increase the effective area by about 5-10 %. This is only relevant for large gratings, because on smaller gratings the rays have a smaller range of blaze angles to begin with.